In [2]:
# Phase 1 (ML Variant - Offline Training): This script uses the historical synthetic data (CSV), land cover map, and the previously generated probability map (.npy) to train an ML model that learns to predict the thermal probability based on context. It then saves the trained model.  

#     Phase 2 (ML Variant - Online Inference): This script runs on the simulated UAV. It loads the trained ML model and uses the UAV's real-time context (position, time, land cover) to predict thermal probability. If the probability is high enough, it triggers the instantiation of average thermal metrics (loaded from the .npz file generated by the original Phase 1 script).

# Assumptions Made:

#     We will train the ML model to predict the thermal presence probability calculated by the original Phase 1 script (i.e., using the values in conditional_probability_map.npy as the target variable). This turns it into a regression problem.
#     We will use a RandomForestRegressor from scikit-learn as an example model. You might substitute other models (like XGBoost, LightGBM, or Neural Networks).
#     Basic feature engineering is used (e.g., using coordinates and context indices directly). More advanced feature engineering might improve results.

Script 1: Phase 1 - Offline ML Model Training -->

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.transform import rowcol
import pyproj
from shapely.geometry import Point, box
from shapely.ops import transform as shapely_transform
import json
import os
import time
from datetime import datetime, timedelta
import math
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib # For saving the model

# --- Configuration ---
CSV_PATH = r"C:\work\ETS_mirza\distributed_montreal\synthetic_thermal_data_5yr_landcover_name.csv" # INPUT: Synthetic Data
GEOJSON_PATH = r"C:\work\ETS_mirza\distributed_montreal\df.geojson" # INPUT: Boundaries (for context)
LAND_COVER_PATH = r"C:\work\ETS_mirza\distributed_montreal\dataset\landcover-2020-classification.tif" # INPUT: Land Cover
METADATA_PATH = r"C:\work\ETS_mirza\distributed_montreal\probability_map_metadata.json" # INPUT: Metadata from original Phase 1
PROB_MAP_PATH = r"C:\work\ETS_mirza\distributed_montreal\conditional_probability_map.npy" # INPUT: Target probabilities

OUTPUT_FOLDER = r"C:\work\ETS_mirza\distributed_montreal"
MODEL_FILENAME = "thermal_predictor_rf_model.joblib" # OUTPUT: Trained Model
MODEL_METADATA_FILENAME = "thermal_predictor_rf_metadata.json" # OUTPUT: Info about model features

# --- Load Metadata (Needed for context mapping and grid info) ---
print(f"Loading metadata from: {METADATA_PATH}")
try:
    with open(METADATA_PATH, 'r') as f:
        metadata = json.load(f)
    grid_crs_str = metadata['grid_crs']
    grid_affine = rasterio.Affine.from_gdal(*metadata['grid_affine_transform_gdal'])
    source_crs_geojson = metadata['grid_origin_in_geojson_crs']['crs']
    season_map = metadata['context_mappings']['season']
    tod_map = metadata['context_mappings']['time_of_day']
    lc_code_to_idx_map = {int(k):v for k,v in metadata['context_mappings']['land_cover_code_to_index'].items()}
    # Also load grid dimensions needed for matching target Y
    grid_height = metadata['grid_dimensions']['height']
    grid_width = metadata['grid_dimensions']['width']
    num_season_cats = len(season_map)
    num_tod_cats = len(tod_map)
    num_lc_cats = len(lc_code_to_idx_map)

except Exception as e:
    print(f"Error loading or processing metadata JSON file: {e}")
    exit()

# --- Load Target Probabilities ---
print(f"Loading target probability map from: {PROB_MAP_PATH}")
try:
    prob_map_target = np.load(PROB_MAP_PATH)
    # Verify shape matches metadata
    if prob_map_target.shape != tuple(metadata['probability_map_shape']):
         print(f"ERROR: Loaded probability map shape {prob_map_target.shape} != metadata shape {metadata['probability_map_shape']}")
         exit()
except Exception as e:
    print(f"Error loading probability map NumPy file: {e}")
    exit()

# --- Load Synthetic Data for Features (X) ---
print(f"Loading synthetic data for features from: {CSV_PATH}")
try:
    thermal_df = pd.read_csv(CSV_PATH, parse_dates=['timestamp_utc'])
    required_cols = ['easting', 'northing', 'land_cover_type'] # Need these for features/target lookup
    thermal_df.dropna(subset=required_cols, inplace=True)

    # Recreate necessary context indices and raster coordinates
    land_cover_names_inv = { # Should match Phase 1 generation script
        "Needleleaf Forest": 1, "Taiga Forest": 2, "Broadleaf Forest": 5, "Mixed Forest": 6,
        "Shrubland": 8, "Grassland": 10, "Shrubland-Lichen-Moss": 11, "Grassland-Lichen-Moss": 12,
        "Barren-Lichen-Moss": 13, "Wetland": 14, "Cropland": 15, "Barren lands": 16,
        "Urban": 17, "Water": 18, "Snow/Ice": 19, "NoData": 0, "Unknown": 0
    }
    default_lc_code = 0
    thermal_df['land_cover_code'] = thermal_df['land_cover_type'].apply(lambda x: land_cover_names_inv.get(x, default_lc_code))

    # Helper functions to get indices
    def get_season_idx(month):
        if month in [6, 7, 8]: return season_map["Summer"]
        if month in [5, 9]: return season_map["Spring/Fall"]
        return None
    def get_tod_idx(hour):
        if 10 <= hour < 12: return time_of_day_map["Morning"]
        if 12 <= hour < 16: return time_of_day_map["Afternoon"]
        if 16 <= hour < 18: return time_of_day_map["Late Afternoon"]
        return None

    thermal_df['season_idx'] = thermal_df['timestamp_utc'].dt.month.apply(get_season_idx)
    thermal_df['tod_idx'] = thermal_df['timestamp_utc'].dt.hour.apply(get_tod_idx)
    thermal_df['lc_idx'] = thermal_df['land_cover_code'].apply(lambda x: lc_code_to_idx_map.get(x))

    # Filter out rows where context is invalid (outside defined time/season/lc)
    thermal_df.dropna(subset=['season_idx', 'tod_idx', 'lc_idx'], inplace=True)
    thermal_df['season_idx'] = thermal_df['season_idx'].astype(int)
    thermal_df['tod_idx'] = thermal_df['tod_idx'].astype(int)
    thermal_df['lc_idx'] = thermal_df['lc_idx'].astype(int)

    # Need coordinates in Raster CRS for grid lookup
    transform_geojson_to_raster = None
    if source_crs_raster != source_crs_geojson:
        transformer_g2r = pyproj.Transformer.from_crs(source_crs_geojson, source_crs_raster, always_xy=True)
        transform_geojson_to_raster = transformer_g2r.transform
    else:
        transform_geojson_to_raster = lambda x, y: (x, y) # Identity transform

    coords_raster = [transform_geojson_to_raster(x, y) for x, y in zip(thermal_df['easting'], thermal_df['northing'])]
    thermal_df['easting_raster'] = [c[0] for c in coords_raster]
    thermal_df['northing_raster'] = [c[1] for c in coords_raster]

    # Get grid indices
    rows, cols = rasterio.transform.rowcol(grid_affine, thermal_df['easting_raster'], thermal_df['northing_raster'])
    thermal_df['grid_row'] = rows
    thermal_df['grid_col'] = cols

    # Filter points outside the defined grid
    valid_grid = (thermal_df['grid_row'] >= 0) & (thermal_df['grid_row'] < grid_height) & \
                 (thermal_df['grid_col'] >= 0) & (thermal_df['grid_col'] < grid_width)
    thermal_df = thermal_df[valid_grid].copy()

    print(f"Prepared {len(thermal_df)} valid events for training features.")

except Exception as e:
    print(f"Error loading or processing CSV for training: {e}")
    exit()


# --- Feature Engineering (Create X) ---
print("Creating feature set (X)...")
# Select features: Location (raster CRS), Time contexts, Land Cover context
# Consider adding cyclical time features for better learning
# Example: Add sin/cos of day of year and time of day
day_of_year = thermal_df['timestamp_utc'].dt.dayofyear
seconds_in_day = thermal_df['timestamp_utc'].dt.hour * 3600 + \
                 thermal_df['timestamp_utc'].dt.minute * 60 + \
                 thermal_df['timestamp_utc'].dt.second

thermal_df['day_sin'] = np.sin(2 * np.pi * day_of_year / 365.0)
thermal_df['day_cos'] = np.cos(2 * np.pi * day_of_year / 365.0)
thermal_df['time_sin'] = np.sin(2 * np.pi * seconds_in_day / (24.0 * 3600.0))
thermal_df['time_cos'] = np.cos(2 * np.pi * seconds_in_day / (24.0 * 3600.0))

# Define feature columns
feature_cols = ['easting_raster', 'northing_raster',
                'season_idx', 'tod_idx', 'lc_idx',
                'day_sin', 'day_cos', 'time_sin', 'time_cos']
X = thermal_df[feature_cols].values
print(f"Feature set X created with shape: {X.shape}")

# --- Target Definition (Create Y) ---
print("Creating target variable (Y) from probability map...")
# Lookup the probability from the loaded map using the calculated indices
y_indices = thermal_df['grid_row'].values
x_indices = thermal_df['grid_col'].values
s_indices = thermal_df['season_idx'].values
t_indices = thermal_df['tod_idx'].values
l_indices = thermal_df['lc_idx'].values

try:
    Y = prob_map_target[y_indices, x_indices, s_indices, t_indices, l_indices]
    print(f"Target variable Y created with shape: {Y.shape}")
except IndexError as e:
    print(f"Error looking up target values in probability map: {e}")
    print("Check consistency between CSV event contexts and probability map dimensions/indices.")
    exit()
except Exception as e:
     print(f"Unexpected error creating target Y: {e}")
     exit()

# --- Train/Test Split ---
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]}, Testing set size: {X_test.shape[0]}")

# --- Model Training ---
# Using RandomForestRegressor as an example
print("Training RandomForestRegressor model...")
# Adjust n_estimators, max_depth etc. for performance/speed
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=20, min_samples_split=10, min_samples_leaf=5)

start_train_time = time.time()
model.fit(X_train, y_train)
end_train_time = time.time()
print(f"Model training finished in {end_train_time - start_train_time:.2f} seconds.")

# --- Evaluation (Optional but Recommended) ---
print("Evaluating model on test set...")
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test Set Mean Squared Error (MSE): {mse:.4g}")
print(f"Test Set R-squared (R2): {r2:.4f}")

# --- Save Model and Feature Info ---
model_output_path = os.path.join(OUTPUT_FOLDER, MODEL_FILENAME)
model_metadata_output_path = os.path.join(OUTPUT_FOLDER, MODEL_METADATA_FILENAME)

print(f"Saving trained model to: {model_output_path}")
try:
    joblib.dump(model, model_output_path)
    print("Model saved successfully.")
except Exception as e:
    print(f"Error saving model: {e}")

model_metadata = {
    "description": "Trained RandomForestRegressor model for predicting P(Thermal|Context)",
    "model_file": MODEL_FILENAME,
    "source_synthetic_data": CSV_PATH,
    "target_probability_map": PROB_MAP_PATH,
    "features": feature_cols,
    "training_timestamp": datetime.now().isoformat(),
    "evaluation_metrics": {"mse": mse, "r2": r2}
}

print(f"Saving model metadata to: {model_metadata_output_path}")
try:
    with open(model_metadata_output_path, 'w') as f:
        json.dump(model_metadata, f, indent=4)
    print("Model metadata saved successfully.")
except Exception as e:
    print(f"Error saving model metadata: {e}")

print("\nML Model Training (Phase 1 - ML Variant) finished.")

#Script 2: Phase 2 - Online ML Inference Simulation

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import rasterio.plot
from rasterio.transform import rowcol
import pyproj
from shapely.geometry import Point, box
from shapely.ops import transform as shapely_transform
import json
import os
import time
from datetime import datetime, timedelta
import math
import warnings
import joblib # For loading model

# --- Configuration ---
OUTPUT_FOLDER = r"C:\work\ETS_mirza\distributed_montreal"
AVG_METRICS_MAP_FILENAME = "average_thermal_metrics.npz" # INPUT: Avg Metrics from original Phase 1
METADATA_FILENAME = "probability_map_metadata.json" # INPUT: Metadata from original Phase 1
LAND_COVER_PATH = r"C:\work\ETS_mirza\distributed_montreal\dataset\landcover-2020-classification.tif" # INPUT: Land Cover Map
REAL_TIME_CSV_PATH = r"C:\work\ETS_mirza\distributed_montreal\real_time.csv" # INPUT: Flight data
MODEL_FILENAME = "thermal_predictor_rf_model.joblib" # INPUT: Trained ML Model
MODEL_METADATA_FILENAME = "thermal_predictor_rf_metadata.json" # INPUT: Model Feature Info

OUTPUT_CSV_FILENAME = "phase2_estimation_output_ml.csv" # OUTPUT: Results

# --- Parameters ---
PROBABILITY_THRESHOLD = 0.5 # Example threshold for ML model output
ENERGY_NEED_THRESHOLD = 80 # Example battery threshold

# --- Load Metadata and Maps ---
METADATA_PATH = os.path.join(OUTPUT_FOLDER, METADATA_FILENAME)
AVG_METRICS_MAP_PATH = os.path.join(OUTPUT_FOLDER, AVG_METRICS_MAP_FILENAME)
MODEL_PATH = os.path.join(OUTPUT_FOLDER, MODEL_FILENAME)
MODEL_METADATA_PATH = os.path.join(OUTPUT_FOLDER, MODEL_METADATA_FILENAME)

print(f"Loading metadata from: {METADATA_PATH}")
try:
    with open(METADATA_PATH, 'r') as f:
        metadata = json.load(f)
    grid_crs_str = metadata['grid_crs']
    # Need affine transform for coordinate conversions if needed by features, or LC lookup
    grid_affine = rasterio.Affine.from_gdal(*metadata['grid_affine_transform_gdal'])
    source_crs_geojson = metadata['grid_origin_in_geojson_crs']['crs']
    season_map = metadata['context_mappings']['season']
    tod_map = metadata['context_mappings']['time_of_day']
    lc_code_to_idx_map = {int(k):v for k,v in metadata['context_mappings']['land_cover_code_to_index'].items()}
    # Need mappings for features
    season_map_inv = {v: k for k, v in season_map.items()}
    tod_map_inv = {v: k for k, v in tod_map.items()}
    lc_idx_to_code_map = {v: k for k, v in lc_code_to_idx_map.items()}

except Exception as e: print(f"Error loading metadata JSON file: {e}"); exit()

print(f"Loading average metrics map from: {AVG_METRICS_MAP_PATH}")
try:
    avg_metrics_data = np.load(AVG_METRICS_MAP_PATH)
    # Verify keys exist
    required_keys = ['avg_lift', 'avg_radius', 'avg_height', 'avg_duration']
    if not all(key in avg_metrics_data for key in required_keys):
        raise ValueError("Missing required keys in average metrics file.")
except Exception as e: print(f"Error loading average metrics NPZ file: {e}"); exit()

print(f"Loading Land Cover Raster: {LAND_COVER_PATH}")
try:
    lc_raster = rasterio.open(LAND_COVER_PATH)
    source_crs_raster = lc_raster.crs
    if str(source_crs_raster) != grid_crs_str:
         print(f"ERROR: Land Cover CRS {source_crs_raster} must match grid CRS {grid_crs_str} from metadata!")
         exit()
except Exception as e: print(f"Error opening land cover file: {e}"); exit()

# --- Load Trained ML Model ---
print(f"Loading trained ML model from: {MODEL_PATH}")
try:
    model = joblib.load(MODEL_PATH)
except Exception as e: print(f"Error loading trained model file: {e}"); exit()

print(f"Loading model metadata from: {MODEL_METADATA_PATH}")
try:
    with open(MODEL_METADATA_PATH, 'r') as f:
        model_metadata = json.load(f)
    feature_cols = model_metadata['features'] # Get feature names used during training
except Exception as e: print(f"Error loading model metadata: {e}"); exit()


# --- Setup CRS Transformations ---
transform_geojson_to_raster = None
transformer_g2r = None
if source_crs_raster != source_crs_geojson:
    try:
        transformer_g2r = pyproj.Transformer.from_crs(source_crs_geojson, source_crs_raster, always_xy=True)
        transform_geojson_to_raster = transformer_g2r.transform
    except Exception as e: print(f"Error setting up transformer: {e}"); exit()
else:
     transform_geojson_to_raster = lambda x, y: (x, y) # Identity

# --- Load Real-Time Flight Data ---
print(f"Loading flight data from: {REAL_TIME_CSV_PATH}")
try:
    flight_df = pd.read_csv(REAL_TIME_CSV_PATH, parse_dates=['timestamp_utc'])
    flight_df.sort_values(by='timestamp_utc', inplace=True)
    # Convert Lat/Lon to projected CRS used for features (Raster CRS)
    flight_gdf = gpd.GeoDataFrame(
        flight_df, geometry=gpd.points_from_xy(flight_df.longitude, flight_df.latitude), crs="EPSG:4326"
    )
    flight_gdf = flight_gdf.to_crs(grid_crs_str) # Convert to Raster CRS
    flight_gdf['easting_raster'] = flight_gdf.geometry.x
    flight_gdf['northing_raster'] = flight_gdf.geometry.y
    print(f"Loaded and transformed {len(flight_gdf)} flight data points.")
except Exception as e: print(f"Error loading/processing real-time CSV: {e}"); exit()

# --- Helper Functions ---
def get_season_idx(month):
    if month in [6, 7, 8]: return season_map["Summer"]
    if month in [5, 9]: return season_map["Spring/Fall"]
    return None

def get_tod_idx(hour):
    if 10 <= hour < 12: return time_of_day_map["Morning"]
    if 12 <= hour < 16: return time_of_day_map["Afternoon"]
    if 16 <= hour < 18: return time_of_day_map["Late Afternoon"]
    return None

# --- Phase 2 Simulation Loop ---
print("Starting Phase 2: Real-time ML estimation simulation...")
start_process_time = time.time()
results = []
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

for index, row in flight_gdf.iterrows():
    current_time = row['timestamp_utc']
    current_easting = row['easting_raster']
    current_northing = row['northing_raster']
    current_battery = row['battery_percent'] # Assumed column name

    # Determine Context Features
    season_idx = get_season_idx(current_time.month)
    tod_idx = get_tod_idx(current_time.hour)

    if season_idx is None or tod_idx is None: # Skip if outside valid time
        results.append({"timestamp_utc": current_time, "ML_detected": False, "comment": "Outside active time"})
        continue

    # Lookup Land Cover Code
    try:
        lc_code_gen = lc_raster.sample([(current_easting, current_northing)], indexes=1)
        lc_code = next(lc_code_gen)[0]
        if lc_code == lc_raster.nodata: lc_code = 0
        lc_idx = lc_code_to_idx_map.get(lc_code)
        if lc_idx is None: lc_idx = lc_code_to_idx_map.get(0) # Default to NoData index if code invalid
    except Exception as e:
        results.append({"timestamp_utc": current_time, "ML_detected": False, "error": f"LC Lookup Error: {e}"})
        continue

    # Construct Feature Vector (matching training order)
    try:
        day_of_year = current_time.dayofyear
        seconds_in_day = current_time.hour * 3600 + current_time.minute * 60 + current_time.second
        day_sin = np.sin(2 * np.pi * day_of_year / 365.0)
        day_cos = np.cos(2 * np.pi * day_of_year / 365.0)
        time_sin = np.sin(2 * np.pi * seconds_in_day / (24.0 * 3600.0))
        time_cos = np.cos(2 * np.pi * seconds_in_day / (24.0 * 3600.0))

        # Ensure order matches 'feature_cols' saved in model metadata
        feature_vector = np.array([[
            current_easting, current_northing,
            season_idx, tod_idx, lc_idx,
            day_sin, day_cos, time_sin, time_cos
        ]]) # Model expects 2D array
    except Exception as e:
        results.append({"timestamp_utc": current_time, "ML_detected": False, "error": f"Feature vector error: {e}"})
        continue

    # Predict Probability using loaded model
    try:
        predicted_prob = model.predict(feature_vector)[0] # Get single prediction
    except Exception as e:
         results.append({"timestamp_utc": current_time, "ML_detected": False, "error": f"Model prediction error: {e}"})
         continue

    # Apply Decision Logic
    thermal_detected = False
    avg_metrics_output = {}
    needs_energy = current_battery < ENERGY_NEED_THRESHOLD
    high_prob = predicted_prob > PROBABILITY_THRESHOLD

    if needs_energy and high_prob:
        thermal_detected = True
        # Lookup Average Metrics using the same context indices
        try:
            # Get grid indices for metrics map lookup
            row_idx, col_idx = rasterio.transform.rowcol(grid_affine, current_easting, current_northing)
            if 0 <= row_idx < avg_metrics_data['avg_lift'].shape[0] and 0 <= col_idx < avg_metrics_data['avg_lift'].shape[1]:
                 metrics_indices = (row_idx, col_idx, season_idx, tod_idx, lc_idx)
                 avg_lift = avg_metrics_data['avg_lift'][metrics_indices]
                 avg_radius = avg_metrics_data['avg_radius'][metrics_indices]
                 avg_height = avg_metrics_data['avg_height'][metrics_indices]
                 avg_duration = avg_metrics_data['avg_duration'][metrics_indices]

                 avg_metrics_output = {
                     "avg_lift_mps": avg_lift if not np.isnan(avg_lift) else None,
                     "avg_radius_m": avg_radius if not np.isnan(avg_radius) else None,
                     "avg_height_m_agl": avg_height if not np.isnan(avg_height) else None,
                     "avg_duration_min": avg_duration if not np.isnan(avg_duration) else None,
                 }
            else:
                 thermal_detected = False # Grid index invalid for metrics map
                 avg_metrics_output = {"error": "Outside metrics grid"}

        except IndexError:
             thermal_detected = False # Index error during metrics lookup
             avg_metrics_output = {"error": "Metrics lookup index error"}
        except Exception as e:
             thermal_detected = False
             avg_metrics_output = {"error": f"Metrics lookup error: {e}"}

    # Store results
    result_row = {
        "timestamp_utc": current_time,
        "uav_lat": row['latitude'],
        "uav_lon": row['longitude'],
        "uav_alt_msl": row['altitude_msl'],
        "uav_battery_percent": current_battery,
        "needs_energy": needs_energy,
        "ML_predicted_prob": predicted_prob,
        "prob_threshold": PROBABILITY_THRESHOLD,
        "ML_detected": thermal_detected,
        # Add location info for context
        "easting_raster": current_easting,
        "northing_raster": current_northing,
        "land_cover_code": lc_code_to_idx_map.get(lc_idx, -1)
    }
    if thermal_detected:
        result_row.update(avg_metrics_output)

    results.append(result_row)

    if (index + 1) % 500 == 0:
        print(f"  Processed {index+1}/{len(flight_gdf)} flight data points...")


print(f"Finished processing {len(flight_gdf)} flight data points.")
end_process_time = time.time()
print(f"Processing took {end_process_time - start_process_time:.2f} seconds.")

# --- Save Output CSV ---
if results:
    output_df = pd.DataFrame(results)
    output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_CSV_FILENAME)
    print(f"Saving estimation results to: {output_path}")
    try:
        output_df.to_csv(output_path, index=False, float_format='%.6g')
        print("Save successful.")
    except Exception as e:
        print(f"Error saving output CSV: {e}")
else:
    print("No results generated.")

# --- Cleanup ---
lc_raster.close()
warnings.resetwarnings()
print("Script finished.")